In [43]:
import os
import time
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from PIL import Image
from pathlib import Path
from skimage.color import rgb2lab, lab2rgb

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("mps")
np.random.seed(123)

In [44]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils import*
from src.models_utils import*

In [45]:
lr_G = 0.0002
lr_d = 0.0002
beta1=0.5
beta2=0.999
lamda = 100.
batch_size = 16
img_size = 256
num_epochs = 5

In [46]:
from fastai.vision.all import *
from torchvision.models.resnet import resnet34
from fastai.vision.models.unet import DynamicUnet


In [47]:
def build_generator(n_input=1, n_output=2, size=img_size):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #device = torch.device("mps")
    backbone = create_body(resnet34(), pretrained=True, n_in=n_input, cut=-2)
    G_net = DynamicUnet(backbone, n_output, (size, size)).to(device)
    return G_net

G_net = build_generator(n_input=1, n_output=2, size=img_size)

In [48]:
G_net = build_generator(n_input=1, n_output=2, size=img_size)
G_net.load_state_dict(torch.load("../models/first-models/generator.pt", map_location=device))

/var/folders/nt/ql8lzvx52bg19b1g9dst1qkr0000gn/T/ipykernel_46074/1577393854.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  G_net.load_state_dict(torch.load("../models/f

<All keys matched successfully>

In [49]:
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 1. Load model
model = Model(G_net=G_net, lr_G=0.0004, lr_D=0.0004, beta1=0.5, beta2=0.999, lamda=100.)
model.load_state_dict(torch.load("../models/first-models/model.pt", map_location=torch.device('cpu')))
model.eval()


/var/folders/nt/ql8lzvx52bg19b1g9dst1qkr0000gn/T/ipykernel_46074/2486336745.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/f

Initializing the model with norm initialization


Model(
  (G_net): DynamicUnet(
    (layers): ModuleList(
      (0): Sequential(
        (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

Colorized photo by path

In [75]:
from skimage import color
# 2. Preprocess image
def get_img_by_path(image_path):
    return Image.open(image_path).convert("RGB")  # Load image in RGB

def preprocess_image(img, img_size=256):
    trans = transforms.Resize((img_size, img_size), Image.BICUBIC)
    img = trans(img)
    img = np.array(img)
    img_to_lab = rgb2lab(img).astype("float32") 
    img_to_lab = transforms.ToTensor()(img_to_lab)
    L = img_to_lab[[0], ...] / 50. - 1. 
    #ab = img_to_lab[[1, 2], ...] / 110. 
    
    return L.unsqueeze(0)

L_tensor = preprocess_image(get_img_by_path("../data/coco2017/test2017/000000000001.jpg"))

L_tensor.shape

torch.Size([1, 1, 256, 256])

In [76]:

# 3. Run inference
with torch.no_grad():
    fake_color = model.G_net(L_tensor)
fake_color.shape

torch.Size([1, 2, 256, 256])

In [77]:
def postprocess_and_save(L_tensor, ab_tensor, save_path="colorized_image.jpg"):
    fake_img = lab_to_rgb(L_tensor, ab_tensor)[0]
    rgb_image = (fake_img * 255).astype(np.uint8)
    rgb_pil = Image.fromarray(rgb_image)
    rgb_pil.save(save_path)
    rgb_pil.show()
    

postprocess_and_save(L_tensor, fake_color)

Colorized video by path

In [74]:
import cv2
import torch
from skimage import color
import numpy as np
from PIL import Image
from torchvision import transforms


def process_frame(frame, img_size, model):
    pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Resize the frame using transforms.Resize
    L_tensor = preprocess_image(pil_frame, img_size)

    # Run inference
    with torch.no_grad():
        fake_color = model.G_net(L_tensor)


    fake_img = lab_to_rgb(L_tensor, fake_color)[0]
    rgb_image = (fake_img * 255).astype(np.uint8)
    
    
    return rgb_image

def to_initial_size(rgb_image, original_size): #dont work correctly
    pil_rgb_image = Image.fromarray(rgb_image)

    # Resize the image back to the original size
    resize_transform = transforms.Resize(original_size, Image.BICUBIC)
    resized_rgb_image = resize_transform(pil_rgb_image)
    resized_rgb_image.show()

    # Convert the resized PIL image back to NumPy format (for OpenCV)
    final_image = np.array(resized_rgb_image)
    return final_image
    

def colorize_video(video_path, model, output_path="colorized_video.mp4"):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    original_size = (width, height)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
    out = cv2.VideoWriter(output_path, fourcc, fps, (256, 256))
    
    while cap.isOpened():
        ret, frame = cap.read()  # Read frame
        if not ret:
            break
        
        # Colorize the frame
        colorized_frame = process_frame(frame, 256, model)

        # Convert RGB to BGR for OpenCV
        #colorized_frame = to_initial_size(colorized_frame, original_size)
        
        # Write the colorized frame to output video
        out.write(cv2.cvtColor(colorized_frame, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR for OpenCV

    # Release everything
    cap.release()
    out.release()
    print(f"Video colorization completed. Saved to {output_path}")

# Example usage
video_path = "../data/videos/inp_v1.mp4"
output_path = "../data/out_videos/colorized_inp_v1.mp4"
colorize_video(video_path, model, output_path)


Video colorization completed. Saved to ../data/out_videos/colorized_inp_v1.mp4
